In [ ]:
import time
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from bs4 import BeautifulSoup
import json

url = "https://www.reddit.com/r/datascience/"
path = "/Users/ajibolaoluwatobiloba/Desktop/AMOD5410- Big Data/Assignments/Assignment 2/chromedriver"
service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)
driver.get(url)

# Wait for the page to load
driver.implicitly_wait(10)

# Scroll down to trigger loading of more articles
scroll_pause_time = 1
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after it's fully loaded
html_text = driver.page_source
soup = BeautifulSoup(html_text, 'lxml')

# Initialize an empty list to store the scraped data
articles_data = []

# Find all the articles on the page and loop through them
articles = soup.find_all('article')
for article in articles:
    # Get the title, url, and author of the article
    title = article['aria-label']
    url = article.find('a', slot='full-post-link')['href']
    author = article.find('span', class_='whitespace-nowrap').text
    
    # Add the article data to the list
    articles_data.append({'title': title, 'url': url, 'author': author})

# Write the scraped data to a JSON file
with open('reddit_scraped.json', 'w') as json_file:
    json.dump(articles_data, json_file, indent=4)

# Close the browser
driver.quit()
